In [15]:
import pandas as pd
import numpy as np
import re
import jaconv

In [11]:
df_train = pd.read_csv('train.csv', index_col='dateid')
df_test = pd.read_csv('test.csv', index_col='dateid')
df_train["TrainFlag"] = True
df_test["TrainFlag"] = False
df = df_train.append(df_test)

In [12]:
def tenki(df):
    weather_main = ['曇', '大雨', '晴', '雨', '快晴', '薄曇']
    weather_sub = ['曇', 'ひょう', '雪', '大雨', '雷', '晴', '雨', 'みぞれ', '薄曇']
    dst = []
    for w in df["weather"]:
        weather_convert = [None] * 15
        w_split = re.split("時々|一時|後|、|を伴う|・", w)
        main = w_split.pop(0)
        weather_convert[weather_main.index(main)] = 1
        while len(w_split) > 0:
            sub = w_split.pop(0)
            if len(sub) > 0:
                weather_convert[weather_sub.index(sub) + 6] = 1
        dst.append(weather_convert)
    for i, h in enumerate(weather_main):
        df[f"main_{h}"] = [d[i] for d in dst]
    for i, h in enumerate(weather_sub):
        df[f"sub_{h}"] = [d[i + 6] for d in dst]
    # return df.drop(["weather"], axis=1)
    return df

def remarks(df):
    idx_pop = None
    for i in range(len(df["remarks"])):
        if type(df["remarks"][i]) != str:
            continue
        if "こだわり" in df["remarks"][i]:
            df["remarks"][i] = "こだわりメニュー"
        elif "イチオシ" in df["remarks"][i]:
            df["remarks"][i] = "イチオシ"
        # elif df["remarks"][i] == "スペシャルメニュー":
        #     idx_pop = df["dateid"][i]
    # if idx_pop is not None:
    #     df = df.drop(idx_pop, axis=0)
    return df

df_tenki = tenki(df)
df_remarks = remarks(df_tenki)
df_remarks.head()
print(df_remarks.columns)

Index(['y', 'week', 'soldout', 'name', 'kcal', 'remarks', 'event', 'payday',
       'weather', 'precipitation', 'temperature', 'TrainFlag', 'main_曇',
       'main_大雨', 'main_晴', 'main_雨', 'main_快晴', 'main_薄曇', 'sub_曇', 'sub_ひょう',
       'sub_雪', 'sub_大雨', 'sub_雷', 'sub_晴', 'sub_雨', 'sub_みぞれ', 'sub_薄曇'],
      dtype='object')


c:\Users\MK\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\MK\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
def save_csv(df, name):
    df_train = df[df["TrainFlag"] == True]
    df_train = df_train.drop(["TrainFlag"], axis=1)
    df_test = df[df["TrainFlag"] == False]
    df_test = df_test.drop(["TrainFlag"], axis=1)
    df_test = df_test.drop(["y"], axis=1)
    df_train.to_csv(f"train_{name}.csv")
    df_test.to_csv(f"test_{name}.csv")

save_csv(df_remarks, "remarks_not_drop")

In [16]:
def bento_group(df):
    bento_ingredient = ["fish", "seafood", "chicken", "pork", "beef", "meat", "vegetable", "rice", "noodle", "other"]
    bento_cooking_method = ["yaki", "ni", "age", "itame", "curry", "other"]
    dst = []
    for n in df["name"]:
        name_convert = [None] * (len(bento_ingredient) +len(bento_cooking_method))
        n = jaconv.kata2hira(n)
        if "ご飯" in n:
            name_convert[bento_ingredient.index("rice")] = 1
        if any(x in n for x in ("麺", "春雨", "そば", "うどん")):
            name_convert[bento_ingredient.index("noodle")] = 1
        if any(x in n for x in ("魚", "ぶり", "鯖", "かれい", "鮭", "さーもん", "かじき", "きす", "たら", "さば", "あじ", "ちゃんちゃん", "ほき")):
            name_convert[bento_ingredient.index("fish")] = 1
        elif any(x in n for x in ("えび", "海老", "いか", "海鮮")):
            name_convert[bento_ingredient.index("seafood")] = 1
        elif any(x in n for x in ("鶏", "ちきん", "鳥", "とり", "唐揚げ", "ぐりる", "親子")):
            name_convert[bento_ingredient.index("chicken")] = 1
        elif any(x in n for x in ("豚", "ぶた", "はむ", "べーこん", "ぽーく", "とんかつ", "ひれかつ", "ささ身", "味噌かつ", "すたみな", "かつ煮")):
            name_convert[bento_ingredient.index("pork")] = 1
        elif any(x in n for x in ("牛", "びーふ", "すき焼き", "すきやき", "めんちかつ", "ろーすかつ", "ぷるこぎ")):
            name_convert[bento_ingredient.index("beef")] = 1
        elif any(x in n for x in ("はんばーぐ", "ろこもこ")):
            name_convert[bento_ingredient.index("pork")] = 1
            name_convert[bento_ingredient.index("beef")] = 1
        elif "ます" in n:
            name_convert[bento_ingredient.index("fish")] = 1
        elif any(x in n for x in ("ころっけ", "ごーや", "菜", "とまと", "枝豆", "なす", "茄子", "筑前煮", "こーん")):
            name_convert[bento_ingredient.index("vegetable")] = 1
        elif any(x in n for x in ("肉", "にく", "ちんじゃお")):
            name_convert[bento_ingredient.index("meat")] = 1
        else:
            name_convert[bento_ingredient.index("other")] = 1
        dst.append(name_convert)

        if any(x in n for x in ("煮", "すき")):
            name_convert[bento_cooking_method.index("ni") + len(bento_ingredient)] = 1
        elif any(x in n for x in ("焼", "やき", "がーりっく", "ぐりる", "すてーき", "はんばーぐ")):
            name_convert[bento_cooking_method.index("yaki") + len(bento_ingredient)] = 1
        elif any(x in n for x in ("揚", "ふらい", "油", "あげ", "かつ", "ころっけ", "天ぷら")):
            name_convert[bento_cooking_method.index("age") + len(bento_ingredient)] = 1
        elif any(x in n for x in ("炒", "そてー")):
            name_convert[bento_cooking_method.index("itame") + len(bento_ingredient)] = 1
        elif "かれー" in n:
            name_convert[bento_cooking_method.index("curry") + len(bento_ingredient)] = 1
        else:
            name_convert[bento_cooking_method.index("other") + len(bento_ingredient)] = 1

    for i, h in enumerate(bento_ingredient):
        df[f"ingredient_{h}"] = [d[i] for d in dst]
    for i, h in enumerate(bento_cooking_method):
        df[f"cook_{h}"] = [d[i + len(bento_ingredient)] for d in dst]
    # return df.drop(["weather"], axis=1)
    return df

df_new = bento_group(df_remarks)
print(df_new.columns)
df_new.head()

Index(['y', 'week', 'soldout', 'name', 'kcal', 'remarks', 'event', 'payday',
       'weather', 'precipitation', 'temperature', 'TrainFlag', 'main_曇',
       'main_大雨', 'main_晴', 'main_雨', 'main_快晴', 'main_薄曇', 'sub_曇', 'sub_ひょう',
       'sub_雪', 'sub_大雨', 'sub_雷', 'sub_晴', 'sub_雨', 'sub_みぞれ', 'sub_薄曇',
       'ingredient_fish', 'ingredient_seafood', 'ingredient_chicken',
       'ingredient_pork', 'ingredient_beef', 'ingredient_meat',
       'ingredient_vegetable', 'ingredient_rice', 'ingredient_noodle',
       'ingredient_other', 'cook_yaki', 'cook_ni', 'cook_age', 'cook_itame',
       'cook_curry', 'cook_other'],
      dtype='object')


,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,...,ingredient_vegetable,ingredient_rice,ingredient_noodle,ingredient_other,cook_yaki,cook_ni,cook_age,cook_itame,cook_curry,cook_other
dateid,,,,,,,,,,,,,,,,,,,,,
2014-04-01,125.0,Tue,1,メンチカツ,377.0,NaN,NaN,NaN,晴時々薄曇,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2014-04-02,99.0,Wed,0,肉じゃが,380.0,NaN,NaN,NaN,薄曇一時晴,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2014-04-03,100.0,Thu,1,酢豚,390.0,NaN,NaN,NaN,大雨,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2014-04-04,90.0,Fri,1,和風ソースハンバーグ,350.0,NaN,NaN,NaN,晴時々曇一時雨、雷を伴う,40.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2014-04-07,107.0,Mon,0,青梗菜牛肉炒め,387.0,NaN,NaN,NaN,晴後一時薄曇,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [17]:
save_csv(df_new, "processed")